In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/ernie/')
init_checkpoint_pt = os.path.join(model_dir, 'ernie_model.bin')
bert_config_file = os.path.join(model_dir, 'ernie_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
# # 初始化构造数据
# from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
# build_pos_train_and_valid_data(vocab_file)



In [3]:
import torch
torch.cuda.set_device(1)

In [ ]:
# 正常训练

from bailian_nlp.modules import BertData

data = BertData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    bert_config_file, 
    init_checkpoint_pt,
    enc_hidden_dim=256,
    freeze=False,
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 50
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


DEBUG:bailian_nlp.web.utils.common:get_data cost 91.99705s


DEBUG:bailian_nlp.web.utils.common:get_data cost 0.188156s


INFO:root:Loading pretrained bert model!
INFO:root:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 513,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 18000
}

INFO:root:Resuming train... Current epoch 0.


In [ ]:
num_epochs = 30
learner.t_total = num_epochs * len(data.train_dl)
learner.fit(num_epochs, target_metric='f1')

In [ ]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 10
# learner.load_model()

base_num_epochs = 30
learner.t_total = base_num_epochs * len(data.train_dl)
# learner.t_total = -1
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


In [ ]:
from bailian_nlp.modules.train import train
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger(config_file)
tagger.init_env(for_train=True)

res = train.validate_step(
    tagger.learner.data.valid_dl, 
    tagger.learner.model,
    tagger.learner.data.id2label,
    tagger.learner.sup_labels
)
print(res)

In [ ]:
tagger.learner.sup_labels

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [ ]:
config_file

In [ ]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger(config_file)


In [ ]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
# text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
# text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
# text = '董事'
# text = '一言不合'
text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
# text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
# text = 'www.baidu.com'
text = '目前服务的主要客户包括玛氏、惠氏、百事食品、箭牌糖果、雀巢、都乐、宝洁、美孚、美泰等国际品牌，协助其实现“互联网+ ”电商化 公司档案 公司名称： 广州淘通'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

In [ ]:
a = 'á'
acc = a[1]

In [ ]:
tokenizer.tokenize(ww)

In [ ]:
ww = '1́.́́未á来́́伙伴机器人费旭锋： 教育机器人市场预计在2025年会达到3000亿'
ww[5]

In [ ]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer
tokenizer = BailianTokenizer()
text = f'1{acc}/m ./w {acc}{acc}未á来{acc}{acc}伙伴机器人/nt 费旭锋/nr ：/w  /w 教育机器人/nz 市场/n 预计/v 在/p 2025/m 年会/n 达到/v 3000亿/m'
tokenizer.tokenize_with_pos_text(text)

In [ ]:
a[6]